In [1]:
from IPython.display import Image
from os import chdir
import numpy as np
import gudhi as gd
import matplotlib.pyplot as plt

In [2]:
# read the file
file_name = "example_2d_v2"

x_range = 0
y_range = 0
simplices = []
vertice_id = 1
i = 0
for line in open("./data/"+file_name+".txt"):
    if(line[0]!='#'):
        
        if(i<=1):
            i=i+1
            continue
            
        elif(i==2):
            rank=[] # sizes of generating sets
            for j in line.split():
                rank.append(int(j))
            i=i+1
            
        else:
            block = line.split(';')
            grid = block[0].split()
            grid_x = int(grid[0])
            grid_y = int(grid[1])
            
            if grid_x > x_range:
                x_range = grid_x
            if grid_y > y_range:
                y_range = grid_y
            
            if len(block)>1:
                face = block[1].split()
                for j in range(len(face)):
                    face[j] = int(face[j])
                simplices.append([grid_x,grid_y,face])
                
            else:
                simplices.append([grid_x,grid_y,[vertice_id]])
                vertice_id += 1
            i=i+1   

end_point = [x_range,y_range] #=[2,3]

            
# To verify the data structur
# simplices contain all the information we need
print("simplices:",simplices)
for i in range(len(simplices)):
    simplex = simplices[i]
    face = simplex[2]
    if(len(face)>1):
        print("This a %d-simplex. Its face is"%(len(face)-1),face)
    else:
        print("This a %d-simplex. Its index is"%(len(face)-1),face[0])


simplices: [[2, 2, [1, 2]], [1, 0, [1]], [0, 1, [2]], [0, 3, [3]]]
This a 1-simplex. Its face is [1, 2]
This a 0-simplex. Its index is 1
This a 0-simplex. Its index is 2
This a 0-simplex. Its index is 3


In [3]:
def create_stair(elbow, end_point):
    # stair a path passing through the elbow.
    stair = []
    
    if(elbow[0]==0 or elbow[1]==end_point[1]):
        for i in range(end_point[1]+1):
            stair.append([0,i])
        for j in range(1,end_point[0]+1):
            stair.append([j,i])
    else:
        for i in range(elbow[1]+1):
            stair.append([0,i])   # i=0
        for j in range(elbow[0]):
            stair.append([j+1,i]) # j=0
        for k in range(i+1, end_point[1]+1):
            stair.append([j+1,k]) # k=2,3
        for l in range(j+2, end_point[0]+1):
            stair.append([l,k])
    return stair

def compute_rk(i,j,k,l,barcodes):
# This function computes the value rank(s,t) where s=(i,j) and t = (k,l)
    if(i>k or j>l or (i==k and j==l)):
        return 0
    
    s_1 = i + j
    t_1 = k + l
    rank0_inv = 0
    rank1_inv = 0

    for barcode in barcodes:
        if(barcode[0]==0):
            if(barcode[1][0]<=s_1 and barcode[1][1]>t_1):
                rank0_inv = rank0_inv + 1
        elif(barcode[0]==1):
            if(barcode[1][0]<=s_1 and barcode[1][1]>t_1):
                rank1_inv = rank1_inv + 1

    return rank0_inv
    #return rank1_inv

In [4]:
rank_invariance = np.array(np.zeros((x_range+1,y_range+1,x_range+1,y_range+1)))

st = gd.SimplexTree()
# st.assign_filtration([3], filtration = 0.8)

#Scan all the elbows
for i in range(x_range+1):
    for j in range(y_range+1):
        elbow = [i,j]
        stair = create_stair(elbow, end_point)

        # Build the simplex tree
        for simplex in simplices:
            for grid_id in stair:
                if simplex[0]<=grid_id[0] and simplex[1]<=grid_id[1]:
                    filtration_value = grid_id[0]+grid_id[1]
                    if((st.insert(simplex[2], filtration = filtration_value))==False):
                        st.assign_filtration(simplex[2], filtration = filtration_value)
                    break
        
        st_gen = st.get_filtration() 
        barcodes = st.persistence()
        
        for start_id in range(len(stair)):
            for end_id in range(start_id,len(stair)):
                grid_start = stair[start_id]
                grid_end = stair[end_id]
                [i,j,k,l]=[grid_start[0],grid_start[1],grid_end[0],grid_end[1]]
                rank_invariance[i,j,k,l]=compute_rk(i,j,k,l,barcodes)
                #print("rank(({0},{1}),({2},{3}))={4}".format(i,j,k,l,int(rank_invariance[i,j,k,l])))
                
                
for i in range(x_range+1):
    for j in range(y_range+1):
        for k in range(x_range+1):
            for l in range(y_range+1):
                print("rank(({0},{1}),({2},{3}))={4}".format(i,j,k,l,int(rank_invariance[i,j,k,l])))

        


rank((0,0),(0,0))=0
rank((0,0),(0,1))=0
rank((0,0),(0,2))=0
rank((0,0),(0,3))=0
rank((0,0),(1,0))=0
rank((0,0),(1,1))=0
rank((0,0),(1,2))=0
rank((0,0),(1,3))=0
rank((0,0),(2,0))=0
rank((0,0),(2,1))=0
rank((0,0),(2,2))=0
rank((0,0),(2,3))=0
rank((0,1),(0,0))=0
rank((0,1),(0,1))=0
rank((0,1),(0,2))=1
rank((0,1),(0,3))=1
rank((0,1),(1,0))=0
rank((0,1),(1,1))=1
rank((0,1),(1,2))=1
rank((0,1),(1,3))=1
rank((0,1),(2,0))=0
rank((0,1),(2,1))=1
rank((0,1),(2,2))=1
rank((0,1),(2,3))=1
rank((0,2),(0,0))=0
rank((0,2),(0,1))=0
rank((0,2),(0,2))=0
rank((0,2),(0,3))=1
rank((0,2),(1,0))=0
rank((0,2),(1,1))=0
rank((0,2),(1,2))=1
rank((0,2),(1,3))=1
rank((0,2),(2,0))=0
rank((0,2),(2,1))=0
rank((0,2),(2,2))=1
rank((0,2),(2,3))=1
rank((0,3),(0,0))=0
rank((0,3),(0,1))=0
rank((0,3),(0,2))=0
rank((0,3),(0,3))=0
rank((0,3),(1,0))=0
rank((0,3),(1,1))=0
rank((0,3),(1,2))=0
rank((0,3),(1,3))=2
rank((0,3),(2,0))=0
rank((0,3),(2,1))=0
rank((0,3),(2,2))=0
rank((0,3),(2,3))=2
rank((1,0),(0,0))=0
rank((1,0),(0,1))=0


In [5]:
#Now it's rank0_invariance
np.save("./result/rank0_invariance_for_"+file_name,rank_invariance)